In [2]:
import csv
import ast
from collections import defaultdict
from neo4j import GraphDatabase
from concurrent.futures import ThreadPoolExecutor

In [3]:
username = "neo4j"                                                # Replace with your username
password = "psWY56eXOUhccUjztpdUmnK8kd7E8FA1j4TlUOvEl5c"          # Replace with your password
driver = GraphDatabase.driver("neo4j+s://104e8e94.databases.neo4j.io", auth=(username, password))

## Importing entities onto neo4j

In [4]:
def create_graph(entities_file, cooccurrences_file):
    # Create nodes and relationships in Neo4j
    with driver.session() as session:
        # print("Conenction Established!")
        # # Load entities
        # with open(entities_file, 'r') as f:
        #     csv_reader = csv.DictReader(f)
        #     for row in csv_reader:
        #         print(row)
        #         # Create entity nodes
        #         session.run("""
        #             MERGE (e:Entity {name: $name, type: $type})
        #         """, name=row['entity'], type=row['type'])
                
        # Load co-occurrences
        with open(cooccurrences_file, 'r') as f:
            csv_reader = csv.DictReader(f)
            for row in csv_reader:
                # Create relationships between entities
                session.run("""
                    MATCH (e1:Entity {name: $entity1}), (e2:Entity {name: $entity2})
                    MERGE (e1)-[r:CO_OCCURS_WITH]->(e2)
                    ON CREATE SET r.weight = $weight
                    ON MATCH SET r.weight = r.weight + $weight
                """, entity1=row['entity1'], entity2=row['entity2'], weight=int(row['weight']))

In [5]:
def create_graph_parallel(entities_file, cooccurrences_file, batch_size=1000):
    with driver.session() as session:
        # print("Conenction Established!")
        # # Load entities
        # with open(entities_file, 'r') as f:
        #     csv_reader = csv.DictReader(f)
        #     for row in csv_reader:
        #         print(row)
        #         # Create entity nodes
        #         session.run("""
        #             MERGE (e:Entity {name: $name, type: $type})
        #         """, name=row['entity'], type=row['type'])
        
        with open(cooccurrences_file, 'r') as f:
            csv_reader = csv.DictReader(f)
            batch = []
            for row in csv_reader:
                # Append each row to the batch
                batch.append({
                    'entity1': row['entity1'],
                    'entity2': row['entity2'],
                    'weight': int(row['weight'])
                })

                # Process batch when size reaches batch_size
                if len(batch) == batch_size:
                    _process_batch(session, batch)
                    batch = []  # Reset batch

            # Process remaining rows
            if batch:
                _process_batch(session, batch)

In [6]:
def _process_batch(session, batch):
    # Helper function to process a batch of rows
    session.write_transaction(_write_relationships, batch)

def _write_relationships(tx, batch):
    # Use an UNWIND query to handle multiple rows in a single transaction
    query = """
        UNWIND $rows AS row
        MATCH (e1:Entity {name: row.entity1}), (e2:Entity {name: row.entity2})
        MERGE (e1)-[r:CO_OCCURS_WITH]->(e2)
        ON CREATE SET r.weight = row.weight
        ON MATCH SET r.weight = r.weight + row.weight
    """
    tx.run(query, rows=batch)

In [ ]:
# Specify the file paths
entities_file = "cleaned_entities.csv"
cooccurrences_file = "cleaned_cooccurrences.csv"

# Create the graph
create_graph_parallel(entities_file, cooccurrences_file)

# Close the driver after operations
driver.close()

/tmp/ipykernel_1776215/925813356.py:3: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(_write_relationships, batch)
